## Importing libraries

In [24]:
import torch
from torch.utils.data import Dataset
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from transformers import Trainer, TrainingArguments
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

## Loading the data

In [2]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

df = pd.read_csv(url_data + 'new_df.csv')

In [3]:
y_liberal = pd.read_csv(url_data + 'y_liberal.csv')
y_data = pd.read_csv(url_data + 'y_data.csv')
X_text = pd.read_csv(url_data + 'X_text.csv')

In [4]:
X_text= X_text['Processed'].tolist()
y = y_liberal

In [5]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_text, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

### Creating a class for creating datasets for the model

In [6]:
from torch.utils.data import Dataset
import torch

class TextDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_length):
        self.tokenized_txt = tokenizer(
            text,
            max_length=max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )
        self.labels = torch.tensor(labels)
      
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {
            key: self.tokenized_txt[key][idx] for key in self.tokenized_txt
        }
        item['labels'] = self.labels[idx]
        return item

## The Distilbert model form Hugging Face

In [7]:
Distilbert = 'distilbert-base-uncased-finetuned-sst-2-english'
Roberta = 'roberta-base'
Bert = 'bert-base-uncased'

#Initializing the model and the tokenizer
tokenizer = AutoTokenizer.from_pretrained(Distilbert)
model = AutoModelForSequenceClassification.from_pretrained(Distilbert, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

### Creating datasets

In [9]:
dataset_train = TextDataset(
    X_train, 
    y_train.values, 
    tokenizer, 
    max_length=20
)

dataset_val = TextDataset(
    X_val, 
    y_val.values, 
    tokenizer, 
    max_length=20
)

dataset_test = TextDataset(
    X_test, 
    y_test.values, 
    tokenizer, 
    max_length=20
)

# An example of the test dataset for the distilbert moodel
dataset_test[9]

{'input_ids': tensor([  101,  2329, 15178,  4674,  2954,  3789,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'labels': tensor([1])}

## Training Setup

In [11]:
# create a dir to store model checkpoints
my_dir = 'roberta_checkpoints'

### Setting up the parameters for the Hugging Face Trainer

In [46]:
# Creating a data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# define training arguments
training_args = TrainingArguments(
    output_dir=my_dir,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="epoch",
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='epoch',
    load_best_model_at_end=True,
    learning_rate=0.00002,
    lr_scheduler_type='cosine',
    metric_for_best_model='accuracy'
    
)

In [48]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Training the model

In [50]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.860493,0.744295,0.801450,0.806321,0.796637
2,0.128300,1.090024,0.746888,0.802589,0.811120,0.794235
3,0.128300,1.166000,0.750519,0.808595,0.803797,0.813451
4,0.087300,1.326371,0.748444,0.804040,0.811582,0.796637
5,0.087300,1.403468,0.757261,0.820414,0.787767,0.855885
6,0.060600,1.449873,0.751037,0.811912,0.795088,0.829464
7,0.060600,1.665433,0.725622,0.768693,0.846821,0.703763
8,0.047000,1.445720,0.753631,0.809467,0.811093,0.807846
9,0.036500,1.568065,0.756224,0.812450,0.809865,0.815052
10,0.036500,1.650692,0.759855,0.814874,0.813898,0.815853


TrainOutput(global_step=5640, training_loss=0.04007929650181574, metrics={'train_runtime': 333.8215, 'train_samples_per_second': 539.031, 'train_steps_per_second': 16.895, 'total_flos': 931100926334400.0, 'train_loss': 0.04007929650181574, 'epoch': 20.0})

## Evaluation of the model

In [51]:
test_result = trainer.evaluate(dataset_test)
print(test_result)

{'eval_loss': 1.824212908744812, 'eval_accuracy': 0.7651632970451011, 'eval_f1': 0.8232539992196645, 'eval_precision': 0.8121632024634334, 'eval_recall': 0.8346518987341772, 'eval_runtime': 0.9265, 'eval_samples_per_second': 2082.118, 'eval_steps_per_second': 65.842, 'epoch': 20.0}


In [ ]:
# Save the model to a specified directory
Dir = 'models/Bert/'
model_directory = Dir + "model"
trainer.save_model(model_directory)

# If you also want to save the tokenizer used during training:
tokenizer_directory = Dir + 'tokenizer_dir'
trainer.tokenizer.save_pretrained(tokenizer_directory)